# Deploying ML models with FastAPI and Streamlit

### Setup

In [ ]:
# Spacy Library and dependencies
!pip install spacy
!python -m spacy download en_core_web_sm

# fastapi and uvicorn - creating api and server
!pip install fastapi uvicorn

# ngrok - providing public urls for testing our api
!pip install nest-asyncio pyngrok 

# Streamlit - Turn data scripts into sharable web apps in minutes.
!pip install streamlit

# Install ngrok file locally - will be used later for Streamlit
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

### Import libraries


In [2]:
import spacy

### Let's use a simple entity detector using spacy

In [3]:
def extract_entities(text):
    # Load English tokenizer, tagger, parser, NER and word vectors
    nlp = spacy.load("en_core_web_sm")


    doc = nlp(text)

    # Find named entities, phrases and concepts
    entities = []
    for entity in doc.ents:
        entities.append((entity.text, entity.label_))
    return entities

In [4]:
# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")

extract_entities(text)

[('Sebastian', 'NORP'),
 ('Google', 'ORG'),
 ('2007', 'DATE'),
 ('American', 'NORP'),
 ('Recode', 'ORG'),
 ('earlier this week', 'DATE')]

### Deploy on FastAPI

In [5]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/{text}')
async def extract_entities(text):
# Load English tokenizer, tagger, parser, NER and word vectors
    nlp = spacy.load("en_core_web_sm")


    doc = nlp(text)

    # Find named entities, phrases and concepts
    entities = []
    for entity in doc.ents:
        entities.append((entity.text, entity.label_))
    return entities

In [6]:
# Have to translate spaces to %20 to use as URL argument
translated_input = text.replace(" ", "%20")
print(translated_input)

When%20Sebastian%20Thrun%20started%20working%20on%20self-driving%20cars%20at%20Google%20in%202007,%20few%20people%20outside%20of%20the%20company%20took%20him%20seriously.%20“I%20can%20tell%20you%20very%20senior%20CEOs%20of%20major%20American%20car%20companies%20would%20shake%20my%20hand%20and%20turn%20away%20because%20I%20wasn’t%20worth%20talking%20to,”%20said%20Thrun,%20in%20an%20interview%20with%20Recode%20earlier%20this%20week.


In [7]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

url = ngrok.connect(port=80)
print('Public URL:', url)
print('Sample URL w/ input:', url.data['public_url'] + "/" + translated_input)
nest_asyncio.apply()

uvicorn.run(app, port=80)

INFO:     Started server process [738]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:80 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "http://fa6ad3348c99.ngrok.io" -> "http://localhost:80"
Sample URL w/ input: http://fa6ad3348c99.ngrok.io/When%20Sebastian%20Thrun%20started%20working%20on%20self-driving%20cars%20at%20Google%20in%202007,%20few%20people%20outside%20of%20the%20company%20took%20him%20seriously.%20“I%20can%20tell%20you%20very%20senior%20CEOs%20of%20major%20American%20car%20companies%20would%20shake%20my%20hand%20and%20turn%20away%20because%20I%20wasn’t%20worth%20talking%20to,”%20said%20Thrun,%20in%20an%20interview%20with%20Recode%20earlier%20this%20week.


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [738]


In [8]:
url.data['public_url'] + "/" + translated_input

'http://fa6ad3348c99.ngrok.io/When%20Sebastian%20Thrun%20started%20working%20on%20self-driving%20cars%20at%20Google%20in%202007,%20few%20people%20outside%20of%20the%20company%20took%20him%20seriously.%20“I%20can%20tell%20you%20very%20senior%20CEOs%20of%20major%20American%20car%20companies%20would%20shake%20my%20hand%20and%20turn%20away%20because%20I%20wasn’t%20worth%20talking%20to,”%20said%20Thrun,%20in%20an%20interview%20with%20Recode%20earlier%20this%20week.'

### Deploy on Streamlit

In [9]:
STREAMLIT_CODE = """
import streamlit as st
import spacy

def extract_entities(text):
    # Load English tokenizer, tagger, parser, NER and word vectors
    nlp = spacy.load("en_core_web_sm")


    doc = nlp(text)

    # Find named entities, phrases and concepts
    entities = []
    for entity in doc.ents:
        entities.append((entity.text, entity.label_))
    return entities

user_input = st.text_input("Input text", "")

st.text(extract_entities(user_input))
"""

In [10]:
#!echo {STREAMLIT_CODE} >> spacy_streamlit.py
st_file = open("spacy_streamlit.py", "w")
n = st_file.write(STREAMLIT_CODE)
st_file.close()

In [11]:
!cat spacy_streamlit.py


import streamlit as st
import spacy

def extract_entities(text):
    # Load English tokenizer, tagger, parser, NER and word vectors
    nlp = spacy.load("en_core_web_sm")


    doc = nlp(text)

    # Find named entities, phrases and concepts
    entities = []
    for entity in doc.ents:
        entities.append((entity.text, entity.label_))
    return entities

user_input = st.text_input("Input text", "")

st.text(extract_entities(user_input))


Use the output of this command as the link to your app.

In [12]:
# If it says "list index out of range" just run it again
get_ipython().system_raw('./ngrok http 8501 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://bf3f4801f4a2.ngrok.io


In [13]:
!streamlit run spacy_streamlit.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.243.172.187:8501

  Stopping...
Traceback (most recent call last):
  File "/usr/local/bin/streamlit", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/streamlit/cli.py", line 233, in main_run
    _main_run(targe

Copy this "Apple is looking at buying U.K. startup for $1 billion" and paste it to your streamlit app

### Summary

Learnings:
1. Deploy model through FastAPI and Streamlit.
2. Use ngrok for creating Public domains.
3. Use the google colab to create temporary backend server.

Applications:
1. Sharing your models within the team.
2. Testing your models.
3. For creating data science apps in development and production stage.
4. For your Data Science Portfolio.

Links:
1. [FastAPI](https://fastapi.tiangolo.com/)
2. [Streamlit](https://www.streamlit.io/)
3. [Spacy](https://spacy.io/)
4. [Ngrok](https://ngrok.com/)

References:
1. [How To Build and Deploy a Machine Learning Model with FastAPI](https://towardsdatascience.com/how-to-build-and-deploy-a-machine-learning-model-with-fastapi-64c505213857)
2. [Deploying and Hosting a Machine Learning Model with FastAPI and Heroku](https://testdriven.io/blog/fastapi-machine-learning/)